## Quiz 4: Hadoop for Fun and Profit 
Conor Meade\
CS 119

In [31]:
import functools
import sys

### 1. Functional Programming [25 points]

### (1.1) add(), sub(), and ra_sub()

In [49]:
def add(*num_list):
    sum = (functools.reduce(lambda x, y: x+y, num_list))
    return sum

print(add(1, 55, 45))
print(add(0, 1, 1, 2, 3, 5, 8, 13))
print(add(1, 2, 3))

def sub(*num_list):
    difference = (functools.reduce(lambda x, y: x-y, num_list))
    return difference

print(sub(5,1,2))

def ra_sub(*num_list):
    difference = (functools.reduce(lambda x, y: y - x, reversed(num_list)))
    return difference

print(ra_sub(5, 1, 2))
print(ra_sub(5))
print(ra_sub(5, 1, 2, 4, 6))

101
33
6
2
6
5
8


### (1.2) zip()

In [40]:
def zip(*num_sequences):
    zipped_lists = [list(map(lambda s: s[i], num_sequences)) for i in range(len(num_sequences[0]))]
    return zipped_lists

print(zip([1], [2]))
print(zip([1, 2, 3], [4, 5, 6]))
print(zip([1, 2, 3], [4, 5, 6], [7, 8, 9]))

[[1, 2]]
[[1, 4], [2, 5], [3, 6]]
[[1, 4, 7], [2, 5, 8], [3, 6, 9]]


### (1.3) zipwith

In [53]:
def zipwith(func, *num_sequences):
    result = list(map(lambda *args: func(*args), *num_sequences))
    return result

print(zipwith(add, [1, 2, 3], [4, 5, 6]))  # [5, 7, 9]
print(zipwith(add, [1, 2, 3], [4, 5, 6], [1, 1, 1]))
print(zipwith(sub, [1, 2, 3], [4, 5, 6], [1, 1, 1]))

[5, 7, 9]
[6, 8, 10]
[-4, -4, -4]


### (1.4) flatten()

In [62]:
def flatten(*tree):
    flat_list = functools.reduce(lambda acc, node: acc + flatten(*node) if isinstance(node, list) else acc + [node], tree, [])
    # flat_list = functools.reduce(lambda x,y: x+y, tree)
    return flat_list

print(flatten([1, [2, [3, 4], [5, 6], 7], 8, [9, 10]]))
print(flatten([[2, 3, 4], 6, [4, 4], [[1, 2], 3, [4, 7, 99]]]))


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[2, 3, 4, 6, 4, 4, 1, 2, 3, 4, 7, 99]


### (1.5) group_by()

In [69]:
def group_by(func, num_sequences):
    func_dict = {}

    for s in num_sequences:
        k = func(s)
        # print(s)
        # print(k)
        if k in func_dict.keys():
            func_dict[k].append(s)
        else:
            func_dict[k] = [s]
    return func_dict


group_by(len, ["hi", "dog", "me", "bad", "good"])

{2: ['hi', 'me'], 3: ['dog', 'bad'], 4: ['good']}

### 2. Confirming Hadoop Installation [15 points]


### (2.1) Acquire the cluster

![Acquire Cluster](Part2/create_cluster.jpeg)

### (2.2)  Load the data into the master, move the data into HDFS

![Load Data, move data into HDFS](Part2/MoveFilesHDFS.png)

### (2.3)  Without writing any code of your own, verify that you have a good installation of hadoop by running wordcount on five-books. The command is similar to...
![Books Count 1](Part2/books_count_1.png)
![Books Count 2](Part2/books_count_2.png)

### (2.4)  Run wordcount using the provided mapper_noll.py and the default reducer aggregate

![mapper 1](Part2/mapred_1.png)
![mapper 2](Part2/mapred_2.png)
![mapper 3](Part2/mapred_3.png)

### (2.5)  Run wordcount using the provided mapper_noll.py and the provided reducer reducer_noll.py

![mapper 4](Part2/mapred_4.png)
![mapper 5](Part2/mapred_5.png)
![mapper 6](Part2/mapred_6.png)